# TODO

Wie viel Prozent falsch online angezeigt?
Wie viel Prozent der Posts kriegen wir?

In [1]:
from pandas import read_csv
from script.normalizer import normalize

deepdarkcti = read_csv('out/deepdarkcti.csv')
reddit2024 = read_csv('out/reddit2024.csv')
darknet = read_csv('out/darknet.csv')
ransomwatch = read_csv('out/ransomwatch.csv')
ransomfind = read_csv('out/ransomfind.csv')
ransomlook = read_csv('out/ransomlook.csv')
ransomwarelive = read_csv('out/reddit2024.csv') # TODO
ransomfeed = read_csv('out/reddit2024.csv') # TODO

normalize(deepdarkcti, reddit2024, darknet, ransomwatch, ransomfind, ransomlook, ransomwarelive, ransomfeed)

In [9]:
from pandas import concat

all_groups = concat([deepdarkcti, reddit2024, darknet, ransomwatch, ransomfind, ransomlook, ransomwarelive, ransomfeed])
all_groups = all_groups.sort_values(by='Group')
all_groups = all_groups.drop_duplicates()
all_groups

,Group,Link,Available,AvailableLabel
0,0mega,omegalock5zxwbhswbisc42o2q2i54vdulyvtqqbudqous...,True,True
257,0mega,omegalock5zxwbhswbisc42o2q2i54vdulyvtqqbudqous...,False,True
4,0mega,0mega.cc,True,False
3,0mega,kbavsfyafrpsostfrkg2w2f7ttf55sz3pfqmoza3o2t3mh...,False,False
258,0mega,0mega.cc,False,True
...,...,...,...,...
109,xinof,wj3b2wtj7u2bzup75tzhnso56bin6bnvsxcbwbfcuvzpc4...,False,False
1339,yanluowang,jukswsxbh3jsxuddvidrjdvwuohtsy4kxg2axbppiyclom...,False,False
141,zeon,zeonrefpbompx6rwdqa5hxgtp2cxgfmoymlli3azoanisz...,False,False
425,zero tolerance,zhuobnfsddn2myfxxdqtpxk367dqnntjf3kq7mrzdgienf...,False,True


In [ ]:


all_groups = pd.concat([
    deepdarkcti,
    reddit2024,
    darknet,
    ransomwatch,
    ransomfind,
    ransomlook,
    ransomwarelive,
    ransomfeed
])

In [ ]:
def get_tracked_status(only_active, indexer_dict: dict):
    tracking_status = { 'Group': [] }
    for i in indexer_dict.keys():
        tracking_status[i] = []
    
    concatted = all_groups
    if only_active:
        concatted = concatted[concatted['Available']]
    total = concatted['Group'].unique()

    for i in indexer_dict.keys():
        indexer = indexer_dict[i]
        if only_active:
            indexer = indexer[indexer['Available']]
        for group in total:
            tracking_status[i].append((indexer['Group'] == group).any())

    for group in total:
        tracking_status['Group'].append(group)
    
    return pd.DataFrame(tracking_status)

indexers = {
    'deepdarkcti': deepdarkcti,
    'reddit2024': reddit2024,
    'darknet': darknet,
    'ransomwatch': ransomwatch,
    'ransomfind': ransomfind,
    'ransomlook': ransomlook,
    'ransomwarelive': ransomwarelive,
    'ransomfeed': ransomfeed
}

tracking_status_all = get_tracked_status(False, indexers)

groups_all = tracking_status_all['Group'].count()
relative_all = tracking_status_all.sum(numeric_only=True).transform(lambda x: x * 100 / groups_all)

ax_all = relative_all.plot.barh(xlim=[0, 100])
ax_all.bar_label(ax_all.containers[0], fmt="%.3g %%")

print(groups_all)


In [ ]:
import json
import pandas as pd

keywords = [
    "medical",
    "hospital",
    "clinic",
    "insur",
    "dent",
    "patient",
    "recover",
    "doctor",
    "nurs",
    "health",
    "facilit",
    "care",
    "lab",
    "pediat",
    "rehab",
    "krank",
    "mediz",
    "gesund",
]

with open("data/posts.json", "r") as f:
    posts = json.load(f)

df = pd.json_normalize(posts)

filtered = df['post_title'].apply(lambda d: any(keyword in d.lower() for keyword in keywords))

df = df[filtered]

df['discovered'] = pd.to_datetime(df['discovered'])
df['discovered'] = df['discovered'].dt.to_period('M')

df

In [ ]:
from pandas import DataFrame, to_datetime

results_all = { 'Title': [], 'Group': [], 'Date': [] }
results_medical = { 'Title': [], 'Group': [], 'Date': [] }
for post in posts:
    title = post['post_title'].lower()
    results_all['Title'].append(post['post_title'])
    results_all['Group'].append(post['group_name'])
    results_all['Date'].append(post['discovered'])
    for keyword in keywords:
        if keyword in title:
            results_medical['Title'].append(post['post_title'])
            results_medical['Group'].append(post['group_name'])
            results_medical['Date'].append(post['discovered'])
            break

df_all = DataFrame(results_all)

df_all['Date'] = to_datetime(df_all['Date'])
df_all['Month'] = df_all['Date'].dt.to_period('M')
print(len(results_all['Title']))

data_all = df_all['Month'].value_counts()

df_medical = DataFrame(results_medical)

df_medical['Date'] = to_datetime(df_medical['Date'])
df_medical['Month'] = df_medical['Date'].dt.to_period('M')
print(len(results_medical['Title']))


data_medical = df_medical['Month'].value_counts()

all = DataFrame({
    'all': data_all,
    'medical': data_medical,
})
all['part'] = all['medical'] / all['all']
all